# Random Forest Classification

In [1]:
import random
import numpy as np
import pandas as pd
import seaborn as sn
from numpy import std
from numpy import mean
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import confusion_matrix, accuracy_score
%matplotlib inline

## All CpG

In [2]:
%cd Z:\PPMI_Data\Excels\NonMotors\Derived250\GeneCpG\Depression
df = pd.read_csv('DepressionCateg_Methylome_APPRDXFilter.csv')
print(df.shape)
df.head(3)

Z:\PPMI_Data\Excels\NonMotors\Derived250\GeneCpG\Depression
(458, 106981)


,Sentrix,PATNO,HYS,Gender,APPRDX,MDS_Depress,cg20815302,cg25795414,cg00959375,cg07781847,...,cg02293835,cg25106148,cg09135009,cg16616325,cg08020573,cg19102621,cg26302779,cg10377074,cg06750710,ch.14.849914F
0,200991620021_R01C01,3001,2,1,1,0,0.907957,0.868910,0.752844,0.827413,...,0.890264,0.096523,0.088376,0.035056,0.040317,0.620467,0.917245,0.873780,0.809904,0.045885
1,200973410167_R04C01,3007,2,1,1,0,0.918626,0.865257,0.798170,0.823846,...,0.841768,0.056917,0.062589,0.032271,0.033286,0.656780,0.876781,0.891459,0.791871,0.061687
2,200973410167_R05C01,3010,2,1,1,0,0.905297,0.864778,0.769766,0.803124,...,0.842802,0.060373,0.042036,0.020839,0.025395,0.691786,0.899872,0.908775,0.834410,0.060866


In [3]:
df1 = df.drop(['Sentrix','PATNO','HYS','Gender','MDS_Depress'], axis=1)
df1.head(2)

,APPRDX,cg20815302,cg25795414,cg00959375,cg07781847,cg05138383,cg27394794,cg12443990,cg19788329,cg22178513,...,cg02293835,cg25106148,cg09135009,cg16616325,cg08020573,cg19102621,cg26302779,cg10377074,cg06750710,ch.14.849914F
0,1,0.907957,0.868910,0.752844,0.827413,0.771393,0.766283,0.791932,0.785104,0.723989,...,0.890264,0.096523,0.088376,0.035056,0.040317,0.620467,0.917245,0.873780,0.809904,0.045885
1,1,0.918626,0.865257,0.798170,0.823846,0.760193,0.801117,0.772456,0.742722,0.580721,...,0.841768,0.056917,0.062589,0.032271,0.033286,0.656780,0.876781,0.891459,0.791871,0.061687


In [4]:
np.random.seed (1)
dframe1 = pd.DataFrame(columns=['Accuracy', 'Std_Dev'])

for i in range(1, 101):
    print(i)
    df2 = df1.groupby('APPRDX').apply(lambda x: x.sample(100))
    X = df2.drop('APPRDX', axis = 1)
    y = df2['APPRDX']
    random_forest_model = RandomForestClassifier(n_estimators = 100, max_depth = 5, criterion = 'gini')
    scores = cross_val_score(random_forest_model, X, y, cv = 10, n_jobs = -1)
    mean_score = scores.mean()
    print(f'Mean accuracy: {mean_score}')
    new_row = {'Accuracy': mean_score, 'Std_Dev': std(scores)}
    dframe1 = pd.concat([dframe1, pd.DataFrame(new_row, index=[0])], ignore_index=True)


1
Mean accuracy: 0.475
2
Mean accuracy: 0.4600000000000001
3
Mean accuracy: 0.515
4
Mean accuracy: 0.49000000000000005
5
Mean accuracy: 0.49000000000000005
6
Mean accuracy: 0.5049999999999999
7
Mean accuracy: 0.535
8
Mean accuracy: 0.5399999999999999
9
Mean accuracy: 0.555
10
Mean accuracy: 0.48999999999999994
11
Mean accuracy: 0.51
12
Mean accuracy: 0.48
13
Mean accuracy: 0.56
14
Mean accuracy: 0.45500000000000007
15
Mean accuracy: 0.5399999999999999
16
Mean accuracy: 0.5349999999999999
17
Mean accuracy: 0.47000000000000003
18
Mean accuracy: 0.53
19
Mean accuracy: 0.52
20
Mean accuracy: 0.555
21
Mean accuracy: 0.465
22
Mean accuracy: 0.5149999999999999
23
Mean accuracy: 0.5449999999999999
24
Mean accuracy: 0.4600000000000001
25
Mean accuracy: 0.5
26
Mean accuracy: 0.48
27
Mean accuracy: 0.53
28
Mean accuracy: 0.47000000000000003
29
Mean accuracy: 0.48
30
Mean accuracy: 0.47000000000000003
31
Mean accuracy: 0.5899999999999999
32
Mean accuracy: 0.465
33
Mean accuracy: 0.535
34
Mean accu

In [5]:
print(mean(dframe1['Accuracy']))
dframe1.to_csv ("Depression_AllCpG_100RF.csv", index = False)

0.5086499999999999


## RanFor 1000 times. Samples = 50

In [6]:
df2 = df1.drop (['APPRDX'], axis=1)

In [7]:
np.random.seed (1)
dframe1 = pd.DataFrame(columns=['Accuracy', 'Std_Dev'])
for i in range (1,101) :
        print(i)
        df3 = df2.sample(50, axis=1)
        df3 = df3.reindex(['APPRDX', *df3.columns], axis=1).assign(APPRDX=df1['APPRDX'])
        dataset = df3.groupby('APPRDX').apply(lambda x: x.sample(100))
        X = dataset.iloc[:, 1:].values
        y = dataset.iloc[:, 0].values
        random_forest_model = RandomForestClassifier(n_estimators = 100, max_depth = 5, criterion = 'gini')
        scores = cross_val_score(random_forest_model, X, y, cv = 10, n_jobs = -1)
        mean_score = scores.mean()
        print(f'Mean accuracy: {mean_score}')
        new_row = {'Accuracy': mean_score, 'Std_Dev': std(scores)}
        dframe1 = pd.concat([dframe1, pd.DataFrame(new_row, index=[0])], ignore_index=True)


1
Mean accuracy: 0.5650000000000001
2
Mean accuracy: 0.525
3
Mean accuracy: 0.485
4
Mean accuracy: 0.5599999999999999
5
Mean accuracy: 0.495
6
Mean accuracy: 0.5599999999999999
7
Mean accuracy: 0.515
8
Mean accuracy: 0.35
9
Mean accuracy: 0.6
10
Mean accuracy: 0.53
11
Mean accuracy: 0.535
12
Mean accuracy: 0.51
13
Mean accuracy: 0.45499999999999996
14
Mean accuracy: 0.465
15
Mean accuracy: 0.5
16
Mean accuracy: 0.54
17
Mean accuracy: 0.505
18
Mean accuracy: 0.5349999999999999
19
Mean accuracy: 0.525
20
Mean accuracy: 0.5549999999999999
21
Mean accuracy: 0.43
22
Mean accuracy: 0.495
23
Mean accuracy: 0.5900000000000001
24
Mean accuracy: 0.55
25
Mean accuracy: 0.505
26
Mean accuracy: 0.445
27
Mean accuracy: 0.535
28
Mean accuracy: 0.47000000000000003
29
Mean accuracy: 0.51
30
Mean accuracy: 0.5150000000000001
31
Mean accuracy: 0.55
32
Mean accuracy: 0.545
33
Mean accuracy: 0.55
34
Mean accuracy: 0.55
35
Mean accuracy: 0.43
36
Mean accuracy: 0.45
37
Mean accuracy: 0.5
38
Mean accuracy: 0.

In [8]:
print(mean(dframe1['Accuracy']))
dframe1.to_csv ("Depression_Rand100CpG_100RF.csv", index = False)

0.5126000000000002


## RanFor 100 times. Samples = 97

In [9]:
np.random.seed (1)
dframe1 = pd.DataFrame(columns=['Accuracy', 'Std_Dev'])
for i in range (1,101) :
        print(i)
        df3 = df2.sample(97, axis=1)
        df3 = df3.reindex(['APPRDX', *df3.columns], axis=1).assign(APPRDX=df1['APPRDX'])
        dataset = df3.groupby('APPRDX').apply(lambda x: x.sample(100))
        X = dataset.iloc[:, 1:].values
        y = dataset.iloc[:, 0].values
        random_forest_model = RandomForestClassifier(n_estimators = 100, max_depth = 5, criterion = 'gini')
        scores = cross_val_score(random_forest_model, X, y, cv = 10, n_jobs = -1)
        mean_score = scores.mean()
        print(f'Mean accuracy: {mean_score}')
        new_row = {'Accuracy': mean_score, 'Std_Dev': std(scores)}
        dframe1 = pd.concat([dframe1, pd.DataFrame(new_row, index=[0])], ignore_index=True)


1
Mean accuracy: 0.58
2
Mean accuracy: 0.55
3
Mean accuracy: 0.5599999999999999
4
Mean accuracy: 0.58
5
Mean accuracy: 0.505
6
Mean accuracy: 0.52
7
Mean accuracy: 0.515
8
Mean accuracy: 0.4600000000000001
9
Mean accuracy: 0.5700000000000001
10
Mean accuracy: 0.5549999999999999
11
Mean accuracy: 0.5549999999999999
12
Mean accuracy: 0.56
13
Mean accuracy: 0.41500000000000004
14
Mean accuracy: 0.48500000000000004
15
Mean accuracy: 0.51
16
Mean accuracy: 0.53
17
Mean accuracy: 0.505
18
Mean accuracy: 0.5400000000000001
19
Mean accuracy: 0.495
20
Mean accuracy: 0.545
21
Mean accuracy: 0.495
22
Mean accuracy: 0.5299999999999999
23
Mean accuracy: 0.5
24
Mean accuracy: 0.48999999999999994
25
Mean accuracy: 0.52
26
Mean accuracy: 0.4800000000000001
27
Mean accuracy: 0.5650000000000001
28
Mean accuracy: 0.4700000000000001
29
Mean accuracy: 0.485
30
Mean accuracy: 0.54
31
Mean accuracy: 0.505
32
Mean accuracy: 0.535
33
Mean accuracy: 0.5
34
Mean accuracy: 0.45499999999999996
35
Mean accuracy: 0.

In [10]:
print(mean(dframe1['Accuracy']))
dframe1.to_csv ("Depression_Rand97CpG_100RF.csv", index = False)

0.5126


## RanFor Top 50 Boruta 100 times. Samples = 50

In [11]:
df = pd.read_csv('Depression_Boruta_Top50_Data.csv')
print(df.shape)
df.head(2)

(458, 52)


,PATNO,APPRDX,cg18780217,cg21747188,cg04841205,cg00052836,cg07456278,cg12510028,cg19690051,cg21062891,...,cg22534925,cg12220605,cg12683454,cg13976366,cg25911279,cg19371224,cg12600134,cg11485463,cg23026864,cg19928377.1
0,3001,1,0.038530,0.064156,0.729990,0.872534,0.582067,0.029810,0.104141,0.887982,...,0.914514,0.082447,0.023826,0.031021,0.031463,0.751572,0.025254,0.011261,0.074048,0.649511
1,3007,1,0.022502,0.056883,0.790912,0.916062,0.667340,0.035511,0.045867,0.923072,...,0.930051,0.113486,0.028970,0.019411,0.020530,0.855625,0.017894,0.013494,0.091529,0.686080


In [12]:
df1 = df.drop(['PATNO'], axis=1)
df1.head(2)

,APPRDX,cg18780217,cg21747188,cg04841205,cg00052836,cg07456278,cg12510028,cg19690051,cg21062891,cg21096050,...,cg22534925,cg12220605,cg12683454,cg13976366,cg25911279,cg19371224,cg12600134,cg11485463,cg23026864,cg19928377.1
0,1,0.038530,0.064156,0.729990,0.872534,0.582067,0.029810,0.104141,0.887982,0.893820,...,0.914514,0.082447,0.023826,0.031021,0.031463,0.751572,0.025254,0.011261,0.074048,0.649511
1,1,0.022502,0.056883,0.790912,0.916062,0.667340,0.035511,0.045867,0.923072,0.884377,...,0.930051,0.113486,0.028970,0.019411,0.020530,0.855625,0.017894,0.013494,0.091529,0.686080


In [13]:
np.random.seed (1)
dframe1 = pd.DataFrame(columns=['Accuracy', 'Std_Dev'])

for i in range(1, 101):
    print(i)
    df2 = df1.groupby('APPRDX').apply(lambda x: x.sample(100))
    X = df2.drop('APPRDX', axis = 1)
    y = df2['APPRDX']
    random_forest_model = RandomForestClassifier(n_estimators = 100, max_depth = 5, criterion = 'gini')
    scores = cross_val_score(random_forest_model, X, y, cv = 10, n_jobs = -1)
    mean_score = scores.mean()
    print(f'Mean accuracy: {mean_score}')
    new_row = {'Accuracy': mean_score, 'Std_Dev': std(scores)}
    dframe1 = pd.concat([dframe1, pd.DataFrame(new_row, index=[0])], ignore_index=True)


1
Mean accuracy: 0.655
2
Mean accuracy: 0.665
3
Mean accuracy: 0.65
4
Mean accuracy: 0.72
5
Mean accuracy: 0.6900000000000001
6
Mean accuracy: 0.71
7
Mean accuracy: 0.6950000000000001
8
Mean accuracy: 0.6849999999999999
9
Mean accuracy: 0.6449999999999999
10
Mean accuracy: 0.65
11
Mean accuracy: 0.66
12
Mean accuracy: 0.645
13
Mean accuracy: 0.7050000000000001
14
Mean accuracy: 0.6599999999999999
15
Mean accuracy: 0.6599999999999999
16
Mean accuracy: 0.6299999999999999
17
Mean accuracy: 0.64
18
Mean accuracy: 0.6950000000000001
19
Mean accuracy: 0.6599999999999999
20
Mean accuracy: 0.665
21
Mean accuracy: 0.6300000000000001
22
Mean accuracy: 0.67
23
Mean accuracy: 0.6500000000000001
24
Mean accuracy: 0.6649999999999999
25
Mean accuracy: 0.6799999999999999
26
Mean accuracy: 0.655
27
Mean accuracy: 0.76
28
Mean accuracy: 0.6499999999999999
29
Mean accuracy: 0.7
30
Mean accuracy: 0.695
31
Mean accuracy: 0.7
32
Mean accuracy: 0.64
33
Mean accuracy: 0.7150000000000001
34
Mean accuracy: 0.67

In [14]:
print(mean(dframe1['Accuracy']))
dframe1.to_csv ("Depression_Boruta50CpG_100RF.csv", index = False)

0.6776499999999998


## RanFor Top 50 NMI 100 times. Samples = 50

In [15]:
df = pd.read_csv('Depression_NMI_Top50_Data.csv')
print(df.shape)
df.head(2)

(458, 51)


,APPRDX,cg16702987,cg26296001,cg20230572,cg24402323,cg10948748,cg14865303,cg00052836,cg16448286,cg12298429,...,cg10595406,cg26595623,cg00714632,cg21899921,cg10474158,cg12321073,cg23999973,cg15736994,cg10192047,cg25744181
0,1,0.691866,0.877350,0.687517,0.060656,0.183438,0.835223,0.872534,0.906014,0.032801,...,0.906883,0.723680,0.025829,0.032846,0.882391,0.013494,0.038562,0.523965,0.019083,0.026860
1,1,0.713975,0.875859,0.741894,0.024917,0.114013,0.867976,0.916062,0.890996,0.042715,...,0.925698,0.739204,0.023401,0.046905,0.897195,0.009607,0.014727,0.562597,0.022214,0.017928


In [16]:
np.random.seed (1)
dframe1 = pd.DataFrame(columns=['Accuracy', 'Std_Dev'])

for i in range(1, 101):
    print(i)
    df2 = df.groupby('APPRDX').apply(lambda x: x.sample(100))
    X = df2.drop('APPRDX', axis = 1)
    y = df2['APPRDX']
    random_forest_model = RandomForestClassifier(n_estimators = 100, max_depth = 5, criterion = 'gini')
    scores = cross_val_score(random_forest_model, X, y, cv = 10, n_jobs = -1)
    mean_score = scores.mean()
    print(f'Mean accuracy: {mean_score}')
    new_row = {'Accuracy': mean_score, 'Std_Dev': std(scores)}
    dframe1 = pd.concat([dframe1, pd.DataFrame(new_row, index=[0])], ignore_index=True)


1
Mean accuracy: 0.68
2
Mean accuracy: 0.675
3
Mean accuracy: 0.6399999999999999
4
Mean accuracy: 0.6900000000000001
5
Mean accuracy: 0.6199999999999999
6
Mean accuracy: 0.7150000000000001
7
Mean accuracy: 0.705
8
Mean accuracy: 0.685
9
Mean accuracy: 0.675
10
Mean accuracy: 0.63
11
Mean accuracy: 0.6900000000000001
12
Mean accuracy: 0.635
13
Mean accuracy: 0.68
14
Mean accuracy: 0.71
15
Mean accuracy: 0.69
16
Mean accuracy: 0.64
17
Mean accuracy: 0.6900000000000001
18
Mean accuracy: 0.7
19
Mean accuracy: 0.7200000000000001
20
Mean accuracy: 0.7
21
Mean accuracy: 0.665
22
Mean accuracy: 0.7150000000000001
23
Mean accuracy: 0.6950000000000001
24
Mean accuracy: 0.605
25
Mean accuracy: 0.7100000000000001
26
Mean accuracy: 0.6400000000000001
27
Mean accuracy: 0.6950000000000001
28
Mean accuracy: 0.65
29
Mean accuracy: 0.64
30
Mean accuracy: 0.675
31
Mean accuracy: 0.7150000000000001
32
Mean accuracy: 0.62
33
Mean accuracy: 0.675
34
Mean accuracy: 0.71
35
Mean accuracy: 0.68
36
Mean accurac

In [17]:
print(mean(dframe1['Accuracy']))
dframe1.to_csv ("Depression_NMI50CpG_100RF.csv", index = False)

0.6771500000000001


## RanFor Top 50 Boruta + NMI 100 times. Samples = 97

In [18]:
df = pd.read_csv('Depression_Top50sMerge_Data.csv')
print(df.shape)
df.head(2)

(458, 99)


,PATNO,APPRDX,cg18780217,cg21747188,cg04841205,cg00052836,cg07456278,cg12510028,cg19690051,cg21062891,...,cg10595406,cg26595623,cg00714632,cg21899921,cg10474158,cg12321073,cg23999973,cg15736994,cg10192047,cg25744181
0,3001,1,0.038530,0.064156,0.729990,0.872534,0.582067,0.029810,0.104141,0.887982,...,0.906883,0.723680,0.025829,0.032846,0.882391,0.013494,0.038562,0.523965,0.019083,0.026860
1,3007,1,0.022502,0.056883,0.790912,0.916062,0.667340,0.035511,0.045867,0.923072,...,0.925698,0.739204,0.023401,0.046905,0.897195,0.009607,0.014727,0.562597,0.022214,0.017928


In [19]:
df1 = df.drop(['PATNO'], axis=1)
df1.head(2)

,APPRDX,cg18780217,cg21747188,cg04841205,cg00052836,cg07456278,cg12510028,cg19690051,cg21062891,cg21096050,...,cg10595406,cg26595623,cg00714632,cg21899921,cg10474158,cg12321073,cg23999973,cg15736994,cg10192047,cg25744181
0,1,0.038530,0.064156,0.729990,0.872534,0.582067,0.029810,0.104141,0.887982,0.893820,...,0.906883,0.723680,0.025829,0.032846,0.882391,0.013494,0.038562,0.523965,0.019083,0.026860
1,1,0.022502,0.056883,0.790912,0.916062,0.667340,0.035511,0.045867,0.923072,0.884377,...,0.925698,0.739204,0.023401,0.046905,0.897195,0.009607,0.014727,0.562597,0.022214,0.017928


In [20]:
np.random.seed (1)
dframe1 = pd.DataFrame(columns=['Accuracy', 'Std_Dev'])

for i in range(1, 101):
    print(i)
    df2 = df1.groupby('APPRDX').apply(lambda x: x.sample(100))
    X = df2.drop('APPRDX', axis = 1)
    y = df2['APPRDX']
    random_forest_model = RandomForestClassifier(n_estimators = 100, max_depth = 5, criterion = 'gini')
    scores = cross_val_score(random_forest_model, X, y, cv = 10, n_jobs = -1)
    mean_score = scores.mean()
    print(f'Mean accuracy: {mean_score}')
    new_row = {'Accuracy': mean_score, 'Std_Dev': std(scores)}
    dframe1 = pd.concat([dframe1, pd.DataFrame(new_row, index=[0])], ignore_index=True)


1
Mean accuracy: 0.685
2
Mean accuracy: 0.7150000000000001
3
Mean accuracy: 0.66
4
Mean accuracy: 0.7050000000000001
5
Mean accuracy: 0.69
6
Mean accuracy: 0.73
7
Mean accuracy: 0.715
8
Mean accuracy: 0.725
9
Mean accuracy: 0.6649999999999999
10
Mean accuracy: 0.6749999999999999
11
Mean accuracy: 0.685
12
Mean accuracy: 0.64
13
Mean accuracy: 0.675
14
Mean accuracy: 0.715
15
Mean accuracy: 0.6849999999999999
16
Mean accuracy: 0.645
17
Mean accuracy: 0.7100000000000001
18
Mean accuracy: 0.735
19
Mean accuracy: 0.695
20
Mean accuracy: 0.6900000000000001
21
Mean accuracy: 0.64
22
Mean accuracy: 0.6799999999999999
23
Mean accuracy: 0.6849999999999999
24
Mean accuracy: 0.64
25
Mean accuracy: 0.6849999999999999
26
Mean accuracy: 0.7
27
Mean accuracy: 0.76
28
Mean accuracy: 0.675
29
Mean accuracy: 0.675
30
Mean accuracy: 0.705
31
Mean accuracy: 0.7200000000000001
32
Mean accuracy: 0.65
33
Mean accuracy: 0.6950000000000001
34
Mean accuracy: 0.675
35
Mean accuracy: 0.6950000000000001
36
Mean ac

In [21]:
print(mean(dframe1['Accuracy']))
dframe1.to_csv ("Depression_Top50sCpG97_100RF.csv", index = False)

0.6959500000000002
